# Redes recorrentes

Ainda é relevante aprender redes recorrentes (RNNs, LSTMs, GRUs), mas sua

*   Item da lista
*   Item da lista

aplicação diminuiu bastante com a ascensão das redes Transformer. Aqui estão alguns pontos para considerar:  

### 📉 **Por que as RNNs perderam espaço?**  
- **Eficiência**: Transformers processam tokens em paralelo, enquanto RNNs são sequenciais, tornando-as mais lentas para treinar.  
- **Dependência de longo prazo**: RNNs têm dificuldade em capturar dependências de longo prazo, problema que LSTMs/GRUs tentam mitigar, mas ainda não resolvem completamente.  
- **Atuação em NLP**: Modelos baseados em Transformer (BERT, GPT, T5) superaram as RNNs em praticamente todas as tarefas de PLN.  

### 📌 **Ainda vale a pena aprender?**  
Sim, mas com foco mais específico:  
- **Séries temporais**: Modelos baseados em RNNs/LSTMs ainda são usados para previsões financeiras, meteorológicas e outras aplicações onde os Transformers nem sempre são a melhor escolha.  
- **Modelos híbridos**: Algumas arquiteturas combinam CNNs, RNNs e Transformers para tarefas multimodais.  
- **Entendimento conceitual**: Conhecer RNNs ajuda a entender a evolução do Deep Learning e algumas limitações que os Transformers resolveram.  

Se seu foco for NLP, visão computacional ou IA generativa, priorize Transformers. Mas se trabalhar com séries temporais ou sistemas embarcados, RNNs ainda podem ser úteis. 🚀

https://www.kaggle.com/code/taavikalaluka/ai-vs-human-text-pytorch-lstm-cnn-99-8-acc


https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text


In [1]:
corpus = [
    "Ahoy matey! Hand over the treasure or walk the plank!",
    "Shiver me timbers! A giant squid off the starboard bow!",
    "Yo-ho-ho and a bottle of rum!",
    "Avast ye landlubber! Where be the golden doubloons?",
    "Shark-infested waters ahead! Batten down the hatches!",
    "Arrr! The kraken wakes! Man the cannons!",
    "Dead men tell no tales, but live ones sing shanties!",
    "Hoist the Jolly Roger! We sail with the morning tide!"
]

# Pré-processamento

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter

# Tokenização e vocabulário
tokens = [word.lower() for sentence in corpus for word in sentence.split()]
vocab = Counter(tokens)
vocab = sorted(vocab, key=vocab.get, reverse=True)
vocab_size = len(vocab)

# Mapeamento palavra para índice
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}

# Modelo de Linguagem

In [3]:
class PirateLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

# Dataset e DataLoader

In [4]:
class PirateDataset(Dataset):
    def __init__(self, corpus, seq_length=3):
        self.seq_length = seq_length
        self.data = []

        for sentence in corpus:
            tokens = sentence.lower().split()
            indices = [word_to_idx[word] for word in tokens]
            for i in range(len(indices) - self.seq_length):
                self.data.append((
                    torch.tensor(indices[i:i+self.seq_length]),
                    torch.tensor(indices[i+1:i+1+self.seq_length])
                ))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

dataset = PirateDataset(corpus, seq_length=3)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

### **Função do PirateDataset**
O `PirateDataset` organiza o texto tokenizado em **sequências de treinamento** para o modelo. Ele:
1. Divide o texto em **sequências de palavras** (tokens) de comprimento fixo (`seq_length`).
2. Cria pares de **entrada** (contexto) e **alvo** (próxima palavra) para treinar o modelo a prever a próxima palavra com base nas anteriores.

---

### **Exemplo Prático**
Vamos usar a primeira frase do corpus:
```python
"Ahoy matey! Hand over the treasure or walk the plank!"
```

#### Passo a Passo:
1. **Tokenização** (após pré-processamento):
   ```python
   ["ahoy", "matey!", "hand", "over", "the", "treasure", "or", "walk", "the", "plank!"]
   ```

2. **Mapeamento para Índices** (exemplo fictício do vocabulário):
   ```python
   {
       "ahoy": 0,
       "matey!": 1,
       "hand": 2,
       "over": 3,
       "the": 4,
       "...": ...
   }
   ```

3. **Sequências Geradas** (com `seq_length = 3`):
   | Entrada (Input)      | Alvo (Target)       |
   |----------------------|---------------------|
   | `[0, 1, 2]`          | `[1, 2, 3]`         |
   | `[1, 2, 3]`          | `[2, 3, 4]`         |
   | `[2, 3, 4]`          | `[3, 4, 5]`         |
   | `[3, 4, 5]`          | `[4, 5, 6]`         |
   | ... (e assim por diante) | ... |

---

### **Explicação Detalhada**
#### 1. Estrutura Interna do Dataset:
Cada entrada é uma sequência de `seq_length` palavras, e o alvo é a **mesma sequência deslocada 1 posição para frente**. Isso força o modelo a aprender a prever a próxima palavra em cada passo.

#### 2. Por Que Isso Funciona?
- O modelo recebe `[ahoy, matey!, hand]` e tenta prever `[matey!, hand, over]`.
- Na prática, para cada posição na sequência de entrada, o modelo aprende a prever a próxima palavra (autoregressão).

---

### **Exemplo Completo (Código + Saída)**
Suponha que temos o seguinte vocabulário mapeado:
```python
word_to_idx = {
    "ahoy": 0, "matey!": 1, "hand": 2, "over": 3,
    "the": 4, "treasure": 5, "or": 6, "walk": 7, "plank!": 8
}
```

#### Dados Gerados para a Frase:
```python
# Sequência Original (índices):
[0, 1, 2, 3, 4, 5, 6, 7, 4, 8]

# Exemplos de pares (input, target):
# Cada linha é um elemento do dataset:
[
    (tensor([0, 1, 2]), tensor([1, 2, 3])),
    (tensor([1, 2, 3]), tensor([2, 3, 4])),
    (tensor([2, 3, 4]), tensor([3, 4, 5])),
    (tensor([3, 4, 5]), tensor([4, 5, 6])),
    (tensor([4, 5, 6]), tensor([5, 6, 7])),
    (tensor([5, 6, 7]), tensor([6, 7, 4])),
    (tensor([6, 7, 4]), tensor([7, 4, 8]))
]
```

---

### **Visualização do Funcionamento**
Para a entrada `[0, 1, 2]` ("ahoy", "matey!", "hand"):
- O modelo tenta prever `[1, 2, 3]` ("matey!", "hand", "over").

Isso cria um **deslizamento de janela** sobre o texto, garantindo que o modelo aprenda padrões locais e sequenciais do corpus.

---

### **Por Que Usar `seq_length`?**
- Define o **contexto máximo** que o modelo usará para prever a próxima palavra.
- Valores pequenos (ex: 3) focam em padrões locais; valores maiores capturam dependências de longo prazo (mas exigem mais dados).

# Treinamento

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PirateLM(vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 200
for epoch in range(epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        output, _ = model(inputs)
        loss = criterion(output.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()


    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

Epoch 1, Loss: 3.9766
Epoch 2, Loss: 3.7064
Epoch 3, Loss: 3.2192
Epoch 4, Loss: 2.7276
Epoch 5, Loss: 2.0673
Epoch 6, Loss: 1.6430
Epoch 7, Loss: 1.2928
Epoch 8, Loss: 0.5196
Epoch 9, Loss: 0.8460
Epoch 10, Loss: 0.7868
Epoch 11, Loss: 0.3296
Epoch 12, Loss: 0.5313
Epoch 13, Loss: 0.3089
Epoch 14, Loss: 0.1987
Epoch 15, Loss: 0.2380
Epoch 16, Loss: 0.1463
Epoch 17, Loss: 0.2000
Epoch 18, Loss: 0.4452
Epoch 19, Loss: 0.0715
Epoch 20, Loss: 0.0786
Epoch 21, Loss: 0.0669
Epoch 22, Loss: 0.0481
Epoch 23, Loss: 0.2804
Epoch 24, Loss: 0.0424
Epoch 25, Loss: 0.0341
Epoch 26, Loss: 0.0310
Epoch 27, Loss: 0.2695
Epoch 28, Loss: 0.0225
Epoch 29, Loss: 0.0444
Epoch 30, Loss: 0.1813
Epoch 31, Loss: 0.0231
Epoch 32, Loss: 0.0247
Epoch 33, Loss: 0.2408
Epoch 34, Loss: 0.0160
Epoch 35, Loss: 0.0158
Epoch 36, Loss: 0.1778
Epoch 37, Loss: 0.0144
Epoch 38, Loss: 0.0129
Epoch 39, Loss: 0.0131
Epoch 40, Loss: 0.0095
Epoch 41, Loss: 0.2305
Epoch 42, Loss: 0.0121
Epoch 43, Loss: 0.1531
Epoch 44, Loss: 0.01

# Função de completar texto

In [6]:
def complete_text(seed_text, num_words=5, temperature=0.7):
    model.eval()
    words = seed_text.lower().split()

    with torch.no_grad():
        for _ in range(num_words):
            inputs = torch.tensor(
                [word_to_idx[word] for word in words[-3:]]  # Usa contexto de 3 palavras
            ).unsqueeze(0).to(device)

            output, _ = model(inputs)
            probabilities = torch.softmax(output[0, -1] / temperature, dim=0)
            next_idx = torch.multinomial(probabilities, 1).item()

            words.append(idx_to_word[next_idx])

    return " ".join(words).capitalize()

# Testando

In [7]:
# Exemplos de prompts
print(complete_text("Shiver me", num_words=5))        # Ex: "Shiver me timbers! A giant squid off"
print(complete_text("Yo-ho-ho", num_words=3))         # Ex: "Yo-ho-ho and a bottle"
print(complete_text("Avast ye", num_words=4))         # Ex: "Avast ye landlubber! Where be"

Shiver me timbers! a giant squid off
Yo-ho-ho and a bottle
Avast ye landlubber! where be the
